In [ ]:
import dv_processing as dv
import numpy as np
import tensorflow as tf
import cv2 as cv

In [ ]:


# Load MoveNet model (lightweight version is preferred for speed)
movenet = tf.saved_model.load(r"D:\Arbeit\Event Camera\aedattomp4\archive")

# Connect to the camera
capture = dv.io.CameraCapture()
print(f"✅ Connected to: {capture.getCameraName()}")

# Get event resolution
resolution = capture.getEventResolution()
width, height = resolution.width, resolution.height

# Initialize accumulator
event_accumulator = np.zeros((height, width), dtype=np.uint8)

# Display window
cv.namedWindow("Live Pose", cv.WINDOW_NORMAL)

# Define keypoint connections
connections = [
    (0, 1), (0, 2), (1, 3), (2, 4),
    (0, 5), (0, 6), (5, 7), (7, 9),
    (6, 8), (8, 10), (5, 6), (5, 11),
    (6, 12), (11, 12), (11, 13), (13, 15),
    (12, 14), (14, 16)
]

def detect_pose(frame):
    rgb = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
    input_image = tf.image.resize_with_pad(tf.expand_dims(rgb, axis=0), 192, 192)
    input_image = tf.cast(input_image, dtype=tf.int32)
    return movenet.signatures['serving_default'](input_image)['output_0'].numpy()

def draw_keypoints(frame, keypoints):
    h, w, _ = frame.shape
    keypoints = keypoints[0, 0]
    
    for y, x, c in keypoints:
        if c > 0.3:
            cx, cy = int(x * w), int(y * h)
            cv2.circle(frame, (cx, cy), 4, (255, 0, 0), -1)
    
    for a, b in connections:
        y1, x1, c1 = keypoints[a]
        y2, x2, c2 = keypoints[b]
        if c1 > 0.3 and c2 > 0.3:
            pt1 = (int(x1 * w), int(y1 * h))
            pt2 = (int(x2 * w), int(y2 * h))
            cv2.line(frame, pt1, pt2, (0, 255, 0), 2)
    
    return frame


RuntimeError: No compatible device discovered